In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [8]:
# Part 1
data = pd.read_csv('Instagram-Reach.csv')

# Convert to DateTime
data['Date'] = pd.to_datetime(data['Date'])

# Get Days of week and group by week
data['Day'] = data['Date'].dt.dayofweek
data['Day'] = data['Day'].map({0: 'Monday', 1: 'Tuesday', 2: 'Wednesday', 3: 'Thursday', 4: 'Friday', 5: 'Saturday', 6: 'Sunday'})
dataDay = data.groupby('Day')

In [ ]:
# Data Info
data.head() # Checks for first 5 columns
data.isnull().sum()
data.describe()
data.info()

In [20]:
# Line Plot
data.plot('Date', 'Instagram reach', title='Instagram Reach over Time')

In [22]:
# Bar plot
data.plot.bar('Date', 'Instagram reach', title='Instagram Reach over Time')

In [24]:
# Box Plot
data.plot.box(title='Instagram Reach over Time')

In [15]:
# Mean for Weekly Grouped Data
print("Mean: ")
print(data.groupby('Day').mean())
print("\nMedian: ")
print(data.groupby('Day').median())
print("\nStandard Deviation: ")
print(data.groupby('Day').std())


Mean: 
                         Date  Instagram reach
Day                                           
Friday    2022-09-30 00:00:00     46666.849057
Monday    2022-09-29 12:00:00     52621.692308
Saturday  2022-09-27 12:00:00     47374.750000
Sunday    2022-09-28 12:00:00     53114.173077
Thursday  2022-10-02 12:00:00     48570.923077
Tuesday   2022-09-30 12:00:00     54030.557692
Wednesday 2022-10-01 12:00:00     51017.269231

Median: 
                         Date  Instagram reach
Day                                           
Friday    2022-09-30 00:00:00          35574.0
Monday    2022-09-29 12:00:00          46853.0
Saturday  2022-09-27 12:00:00          40012.0
Sunday    2022-09-28 12:00:00          47797.0
Thursday  2022-10-02 12:00:00          39150.0
Tuesday   2022-09-30 12:00:00          48786.0
Wednesday 2022-10-01 12:00:00          42320.5

Standard Deviation: 
                                 Date  Instagram reach
Day                                                   
Frida

In [ ]:
# Bar Plot per Week Day
dataDay.plot.bar(x='Day', y='Instagram reach', title='Instagram Reach over Time')